In [ ]:
# Imports
import os
import bz2
import numpy as np
from PIL import TiffImagePlugin
from IPython.core.display import display
from PIL import Image as Img

In [ ]:
import sys
sys.path.append("../")
from algorithms.flow import flow
from data_structures.location_graph import LocationGraph

In [ ]:
def load_image_data() -> TiffImagePlugin.TiffImageFile:
    path = os.path.dirname(os.path.abspath('')) + '/data/'
    data = bz2.decompress(open(path + 'dem.tif.bz2', 'rb').read())
    with open(path + 'dem2.tiff', 'wb') as handle:
        handle.write(data)
    return Img.open(path + 'dem2.tiff')

In [ ]:
def load_and_crop_data():
    raw_height_array = np.asarray(load_image_data())
    print(type(raw_height_array))
    print(raw_height_array.max())
    print(raw_height_array.min())
    scaled_data=raw_height_array
    # scaled_data = (raw_height_array/(raw_height_array.max()/255.0)).astype(np.uint8)
    crop_size = 1000
    offset = 900
    return scaled_data[offset:offset+crop_size, offset:offset+crop_size]
cropped_data = load_and_crop_data()

In [ ]:
%%time
height_map = cropped_data.tolist()
graph = LocationGraph(height_map)

In [ ]:
class ImageWriter(object):
    def __init__(self, path, size, save_frequency=10000):
        self.write_path = path
        self.index = 0
        self.image = Img.new('L', (size, size))
        self.pixels = self.image.load()
        self.save_frequency=save_frequency
        self.image_number = 0

    def update(self, node):
        self.index += 1
        
        self.pixels[node.home[0],node.home[1]] = 100
        if self.index % self.save_frequency == 0:
            filename = f"{self.write_path}{self.image_number}.tiff"
            (self.image).save(filename)
            print(f"writer viewed {self.index} nodes")
            print(f"saved image {filename}")
            self.image_number += 1

# Initialize image file writer
def initialize_writer():
    writer_path = f"{os.path.dirname(os.path.dirname(os.path.abspath('')))}/data/img/"
    
    print(f"Writer will save images to {writer_path}")
    writer = ImageWriter(writer_path, 1000, 5000)
    return writer
writer = initialize_writer()

In [ ]:
%%time
nodes_with_flow = flow(graph, writer)